In [55]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import seaborn as sns
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv


In [11]:
df = pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv",index_col="id")

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5110 entries, 9046 to 44679
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 479.1+ KB


In [13]:
df.fillna(df.bmi.mean(),inplace=True)

In [17]:
df1 = df.iloc[:,[0,2,3,4,5,6,9]]
df2 = df.iloc[:,[1,7,8]]
y = df.iloc[:,[-1]]

In [18]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
ohe = OneHotEncoder()
sc = StandardScaler()

In [19]:
df1 = pd.DataFrame(ohe.fit_transform(df1).toarray())
df2 = pd.DataFrame(sc.fit_transform(df2))

In [21]:
df = pd.concat([df1,df2],axis=1)
df.columns = [a for a in range(0,len(df.columns))]

In [34]:
from sklearn.model_selection import train_test_split
xtr,xte,ytr,yte = train_test_split(df,y,test_size=0.25)

In [43]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
model = Sequential()
model.add(Dense(512,activation='relu',input_shape=(23,)))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(loss="binary_crossentropy", optimizer=Adam(lr=0.0001), metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 512)               12288     
_________________________________________________________________
dense_28 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_30 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_16 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 128)              

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [44]:
from keras.callbacks import EarlyStopping
cb = EarlyStopping(
    monitor='accuracy',
    min_delta=0.001,
    patience=100,
    mode='auto')

In [45]:
model.fit(xtr,ytr,epochs=1000,batch_size=10000,validation_split=0.25,callbacks=cb)

Epoch 1/1000
1/1 [==============================] - 1s 1s/step - loss: 0.7218 - accuracy: 0.3511 - val_loss: 0.6919 - val_accuracy: 0.6054
Epoch 2/1000
1/1 [==============================] - 0s 143ms/step - loss: 0.7096 - accuracy: 0.4019 - val_loss: 0.6809 - val_accuracy: 0.9144
Epoch 3/1000
1/1 [==============================] - 0s 145ms/step - loss: 0.6942 - accuracy: 0.5070 - val_loss: 0.6710 - val_accuracy: 0.9415
Epoch 4/1000
1/1 [==============================] - 0s 144ms/step - loss: 0.6834 - accuracy: 0.5710 - val_loss: 0.6617 - val_accuracy: 0.9426
Epoch 5/1000
1/1 [==============================] - 0s 152ms/step - loss: 0.6708 - accuracy: 0.6569 - val_loss: 0.6523 - val_accuracy: 0.9426
Epoch 6/1000
1/1 [==============================] - 0s 144ms/step - loss: 0.6593 - accuracy: 0.7272 - val_loss: 0.6423 - val_accuracy: 0.9426
Epoch 7/1000
1/1 [==============================] - 0s 146ms/step - loss: 0.6453 - accuracy: 0.7978 - val_loss: 0.6314 - val_accuracy: 0.9426
Epoch 8/1

In [61]:
model.evaluate(xte,yte)

40/40 [==============================] - 0s 2ms/step - loss: 0.1468 - accuracy: 0.9570


[0.1468483805656433, 0.9569640159606934]